----------------------------------------------------------------------------------------------------------------------

In [1]:
%pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import requests
import time
import pyttsx3
import random
import threading
import torch
import json
import threading

from datetime import datetime, timedelta
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.llms import HuggingFacePipeline
from transformers import pipeline as hf_pipeline
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS

c:\Users\Hp\.conda\envs\AI_Agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------------------------------------------------------------------------------------------------------------------

In [3]:
# ------------------------------------------------------------
# 1) Çevre değişkenlerini yükle ve temel ayarlar
load_dotenv()
STUDENT_ID = "ogrenci_001"

# Google API key kontrolü
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    # Eğer anahtar tanımlı değilse hata ver
    raise ValueError("GOOGLE_API_KEY bulunamadı! .env dosyasını kontrol edin.")

# Gemini LLM ve embedding yükleme
# Bazı durumlarda GoogleGenerativeAI veya embedding yüklemesi hata verebilir.
# İlk denemede başarısız olursa, alternatif yükleme sırasında embedding de tekrar yüklenir.
try:
    LLM = GoogleGenerativeAI(model="gemini-1.5-flash-8b-001", temperature=0.1)
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    print("✅ Google AI modelleri başarıyla yüklendi")
except Exception as e:
    # İlk deneme başarısız olduysa, hata mesajını yaz ve alternatif model dene.
    print(f"❌ Google AI model yükleme hatası: {e}")
    print("🔄 Alternatif model deneniyor...")
    try:
        LLM = GoogleGenerativeAI(model="gemini-1.5-flash-8b-001", temperature=0.1)
        # Alternatif yükleme sırasında embedding'i de yeniden tanımla
        embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        print("✅ Alternatif model ve embedding yüklendi")
    except Exception as e2:
        # Eğer yine hata alırsak, hatayı yükselt
        print(f"❌ Alternatif model de başarısız: {e2}")
        raise

# ChromaDB başlatma
try:
    vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)
    print("✅ ChromaDB başarıyla başlatıldı")
except Exception as e:
    print(f"❌ ChromaDB başlatma hatası: {e}")
    import shutil
    # Eğer klasör bozulduysa, temizleyip yeniden oluştur.
    try:
        shutil.rmtree("student_memory", ignore_errors=True)
        vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)
        print("✅ ChromaDB yeniden oluşturuldu")
    except Exception as e2:
        print(f"❌ ChromaDB yeniden oluşturma hatası: {e2}")
        raise

# QA Chain oluşturma
try:
    qa_chain = RetrievalQA.from_chain_type(
        llm=LLM,
        retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
        chain_type="stuff"
    )
    print("✅ QA Chain başarıyla oluşturuldu")
except Exception as e:
    print(f"❌ QA Chain oluşturma hatası: {e}")
    raise

✅ Google AI modelleri başarıyla yüklendi


C:\Users\Hp\AppData\Local\Temp\ipykernel_22408\573029798.py:35: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)


✅ ChromaDB başarıyla başlatıldı
✅ QA Chain başarıyla oluşturuldu


----------------------------------------------------------------------------------------------------------------------

In [4]:
# 2) Dış API uç noktalarını ve küçük modeli yükle

ENDPOINT_ATTENTION = "http://127.0.0.1:8001/attention"
ENDPOINT_SCRIPT = "http://localhost:5002/get_texts"
ENDPOINT_KYBRD_MOUSE_INTERRUPT = "http://localhost:5001/api/status"

# accelerate modülünü yükle (GPU/CPU ayarı için)
import accelerate  # noqa: F401  # Yüklendiğinden emin olmak için

# Cihaz belirleme
device = 0 if torch.cuda.is_available() else -1

# HuggingFace Pipeline ile küçük model
small_pipe = hf_pipeline(
    "text-generation",
    model="microsoft/DialoGPT-small",
    trust_remote_code=True,
    device=device,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    max_new_tokens=256,
)
small_llm = HuggingFacePipeline(pipeline=small_pipe)
print("✅ Küçük model başarıyla yüklendi")

# Sayaçlar ve parametreler (şimdilik kullanılmıyor ama ileride gerekebilir)
iteration_count = 0
last_summary_iter = 0
SUMMARY_INTERVAL = 5


Device set to use cpu


✅ Küçük model başarıyla yüklendi


C:\Users\Hp\AppData\Local\Temp\ipykernel_22408\2317526927.py:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  small_llm = HuggingFacePipeline(pipeline=small_pipe)


----------------------------------------------------------------------------------------------------------------------

In [5]:
# 3) Uyarı ve analiz fonksiyonları

active_tools_lock = threading.Lock()
active_tools: set[str] = set()

def track_tool_call(func, name):
    def wrapper(*args, **kwargs):
        with active_tools_lock:
            active_tools.add(name)
        try:
            return func(*args, **kwargs)
        finally:
            with active_tools_lock:
                active_tools.discard(name)
    return wrapper

def summarize_records_with_small_llm(records):
    """Bir liste halindeki sensör kayıtlarını küçük LLM ile anlamlı şekilde özetler."""
    if not records:
        return "Son dakikada kaydedilmiş sensör verisi yok."
    # Her kaydı küçük modele madde madde özetlet
    chunks = [json.dumps(r, ensure_ascii=False, default=str) for r in records]
    prompt = (
        "Aşağıda son 1 dakikadaki sensör kayıtları yer alıyor.\n"
        "Her bir kaydı kısa ve anlaşılır şekilde madde madde özetle, önemli noktaları çıkar:\n\n"
        + "\n".join(chunks)
    )
    summary = small_llm(prompt)
    # HuggingFacePipeline çıktısı genelde liste oluyor
    summary_text = summary[0]['generated_text'] if isinstance(summary, list) else str(summary)
    return summary_text.strip()



def dikkat_uyarisi_ver(_):
    """En son dikkat verisini çekerek ekrana uyarı yazdırır."""
    try:
        recent = vectordb.similarity_search("attention", k=1)[0]
        data = recent.metadata
        if not data.get("screen", False) or data.get("attention", 0) < 0.5:
            print("⚠️ Dikkatin dağıldı! Lütfen odaklan.")
            print("\033[91m[Çerçeve RENGİ: KIRMIZI - ODAK YOK]\033[0m")
    except Exception as e:
        print(f"🚨 Dikkat modülü hatası: {e}")


def mola_onerisi_chromadb(_):
    """Son 10 dikkat kaydına bakarak mola önerisi verir."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        att_20s = [
            doc.metadata.get("att_20min", 0)
            for doc in recent_docs
            if isinstance(doc.metadata.get("att_20min"), (float, int))
        ]
        if len(att_20s) >= 3 and sum(att_20s) / len(att_20s) < 0.6:
            print("😴 20dk ortalaman düşük. Mola vermelisin.")
            print("\033[95m[Mola Önerisi: Düşük dikkat algılandı.]\033[0m")
    except Exception as e:
        print(f"🚨 Mola modülü hatası: {e}")


def periyot_onerisi_llm_ileri(_):
    """LLM kullanarak öğrenme stratejisi önerisi üretir."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=15)
        att_vals = [doc.metadata.get("attention", 0) for doc in recent_docs]
        focus_vals = [doc.metadata.get("focus_score", 0) for doc in recent_docs]
        attention_summary = "\n".join([
            f"- Dikkat: {round(doc.metadata.get('attention', 0), 2)} "
            f"| 20dk: {round(doc.metadata.get('att_20min', 0), 2)} "
            f"| Focus: {round(doc.metadata.get('focus_score', 0), 2)}"
            for doc in recent_docs
        ])
        if len(att_vals) < 5:
            print("🔁 Kişisel öneri için yeterli veri yok.")
            return

        prompt = f"""
🧠 Öğrenci dikkat geçmişi:
{attention_summary}

📊 Ortalama dikkat: {round(sum(att_vals)/len(att_vals), 2)}
🎙️ Ortalama focus: {round(sum(focus_vals)/len(focus_vals), 2)}

Bu öğrenciye uygun öğrenme periyodu ve stratejileri öner.
"""
        feedback = qa_chain.run(prompt)
        print("\n🧠 Öğrenme Önerisi:\n", feedback)
    except Exception as e:
        print(f"🚨 Öğrenme önerisi hatası: {e}")


def zihin_yorgunlugu_tahmini(_):
    """Dikkat verilerindeki düşüşe göre zihin yorgunluğunu tespit eder."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=10, filter={"type": "sensor"})
        if len(recent_docs) < 5:
            print("🔁 Yorgunluk analizi için yeterli veri yok.")
            return

        attention_values = [doc.metadata.get("attention", 0) for doc in recent_docs]
        # İlk üç ve son üç ölçümü ortalama alarak farkı hesapla
        first_half_avg = sum(attention_values[:3]) / 3
        last_half_avg = sum(attention_values[-3:]) / 3
        delta = first_half_avg - last_half_avg

        if delta > 0.15:
            print("🧠 Zihin yorgunluğu tespit edildi. Dikkatin ciddi oranda düştü.")
            print("\033[93m[Yorgunluk Algılandı: Kısa bir ara vermelisin.]\033[0m")
    except Exception as e:
        print(f"🚨 Yorgunluk tahmin modülü hatası: {e}")


def vak_ogrenme_tarzi_tahmini(_):
    """VAK öğrenme tarzını tahmin etmeye çalışır."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        visual = auditory = kinesthetic = 0

        for doc in recent_docs:
            m = doc.metadata
            if m.get("screen"):
                visual += 1
            if m.get("focus_score", 0) and m.get("focus_score", 0) > 0.6:
                auditory += 1
            if m.get("keyboard_activity") or m.get("mouse_activity"):
                kinesthetic += 1

        toplam = visual + auditory + kinesthetic
        if toplam == 0:
            print("🔁 Öğrenme tarzı için yeterli veri yok.")
            return

        style = max(
            (visual, "Görsel"),
            (auditory, "İşitsel"),
            (kinesthetic, "Kinestetik"),
        )[1]
        print(f"📚 Tahmini öğrenme tarzın: \033[94m{style}\033[0m")
    except Exception as e:
        print(f"🚨 Öğrenme tarzı modülü hatası: {e}")


def oturum_ozet_raporu(_):
    """Son verilerle oturum özeti sunar."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        if not recent_docs:
            print("🔁 Özet için yeterli veri yok.")
            return

        avg_att = sum(doc.metadata.get("attention", 0) for doc in recent_docs) / len(recent_docs)
        avg_focus = sum(doc.metadata.get("focus_score", 0) for doc in recent_docs) / len(recent_docs)
        en_focused = max(recent_docs, key=lambda d: d.metadata.get("focus_score", 0))
        saat = en_focused.metadata.get("timestamp", "bilinmiyor")

        print("\n📊 \033[92mOturum Özeti:\033[0m")
        print(f"- Ortalama dikkat: {round(avg_att, 2)}")
        print(f"- Ortalama focus: {round(avg_focus, 2)}")
        print(f"- En odaklı zaman: {saat}")
    except Exception as e:
        print(f"🚨 Oturum özeti hatası: {e}")


def sesli_motivasyon_ver(mesaj="Harika gidiyorsun! Odaklanmaya devam et."):
    """Basit sesli motivasyon mesajı verir."""
    try:
        engine = pyttsx3.init()
        engine.setProperty('rate', 175)
        engine.say(mesaj)
        engine.runAndWait()
    except Exception as e:
        print(f"[Sesli Uyarı Hatası] {e}")


def analiz_sorudan_anlam_cikar_ve_yanitla(soru: str):
    """Soruyu ve son dikkat verilerini kullanarak cevabı üretir."""
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        attention_lines = "\n".join([
            f"- Dikkat: {round(doc.metadata.get('attention', 0), 2)} "
            f"| Focus: {round(doc.metadata.get('focus_score', 0), 2)}"
            for doc in recent_docs
        ])
        prompt = f"""
Kullanıcının sorusu: \"{soru}\"

Aşağıda kullanıcının son dikkat ve etkileşim verileri var:
{attention_lines}

Bu verilerle birlikte, kullanıcının sorusuna cevap ver:
- Soruyu anlamlandır.
- Eğer performans sorusuysa analiz yap.
- Yorgunluksa belirt.
- Motivasyon istiyorsa cesaretlendir.
- Öğrenme önerisi istiyorsa tavsiye ver.
- Konuşma tonun sıcak ve koçvari olsun.
"""
        yanit = LLM.invoke(prompt)
        # Bazı durumlarda yanıt nesne dönebilir, içeriği düz metne çevirmeyi dene.
        if isinstance(yanit, dict) and "text" in yanit:
            return yanit["text"]
        return str(yanit)
    except Exception as e:
        return f"[Analiz Hatası] {e}"


def ozet_ses_logs(_):
    """
    Kayıt dosyasındaki ham verileri küçük modellerle özetler,
    sonra Gemini üzerinden daha akıllı bir final özet çıkararak PDF'e kaydeder.
    """
    log_path = "system_session_full.txt"
    try:
        with open(log_path, "r", encoding="utf-8") as f:
            text = f.read()
    except Exception as e:
        return f"❌ Log dosyası okunamadı: {e}"

    # Metni 2000 karakterlik parçalara böl
    chunks = [text[i:i + 2000] for i in range(0, len(text), 2000)]

    # Küçük modelle her parçayı özetle
    small_summaries = []
    for chunk in chunks:
        small_prompt = (
            f"Aşağıdaki zaman damgalı ses kaydını kısaca madde madde özetle:\n\n{chunk}"
        )
        try:
            summary = small_llm(small_prompt)
            # HuggingFacePipeline sonucu bir liste veya string olabilir; stringe çevir
            summary_text = summary[0]['generated_text'] if isinstance(summary, list) else str(summary)
            small_summaries.append(summary_text.strip())
        except Exception as sm_e:
            small_summaries.append(f"[Özetlenemedi: {sm_e}]")

    # Küçük özetleri birleştir
    combined_small_summary = "\n\n".join(small_summaries)

    # Birleştirilmiş özeti Gemini’ye gönder, detaylı bir final özet al
    final_prompt = f"""
Aşağıda, kayıtların küçük modeller tarafından çıkarılmış özetleri var:
{combined_small_summary}

lütfen bu özetleri kullanarak:
- Metnin anahtar noktalarını net ve sıralı şekilde maddele,
- Gerektiğinde örnekler veya bağlam ekle,
- “Akıllıca” birleştirilmiş, akıcı bir final özeti üret.
"""
    try:
        detailed_summary = qa_chain.run(final_prompt)
    except Exception as gm_e:
        return f"❌ Gemini ile özet çıkarmada hata: {gm_e}"

    # PDF oluştur
    pdf_path = "session_summary.pdf"
    try:
        c = canvas.Canvas(pdf_path, pagesize=A4)
        width, height = A4
        text_obj = c.beginText(40, height - 40)
        text_obj.setFont("Helvetica", 11)
        for line in detailed_summary.split("\n"):
            text_obj.textLine(line)
            # Her sayfada satır limiti kontrolü
            if text_obj.getY() < 40:
                c.drawText(text_obj)
                c.showPage()
                text_obj = c.beginText(40, height - 40)
                text_obj.setFont("Helvetica", 11)
        c.drawText(text_obj)
        c.save()
    except Exception as e:
        return f"❌ PDF oluşturulurken hata: {e}"

    return f"✅ Özet PDF hazır: {pdf_path}"



In [6]:
# 4) Araçları tanımla ve agent başlat

tools = [
    Tool(name="DikkatUyarisi", func=dikkat_uyarisi_ver, description="Dikkat seviyesi düşükse uyarı verir."),
    Tool(name="MolaOnerisi", func=mola_onerisi_chromadb, description="20dk ortalama dikkat düşüklüğünde mola önerir."),
    Tool(name="OgrenmePeriyoduOnerisi", func=periyot_onerisi_llm_ileri, description="Dikkat geçmişine göre öğrenme stratejisi önerir."),
    Tool(name="ZihinYorgunluguTahmini", func=zihin_yorgunlugu_tahmini, description="Son dikkat verilerindeki düşüşe göre zihin yorgunluğunu tespit eder."),
    Tool(name="OgrenmeTarziTahmini", func=vak_ogrenme_tarzi_tahmini, description="Kullanıcının VAK öğrenme tarzını tahmin eder."),
    Tool(name="OturumOzeti", func=oturum_ozet_raporu, description="Son verilerle oturum özeti sunar."),
    Tool(name="SoruyaGoreAnalizYap", func=analiz_sorudan_anlam_cikar_ve_yanitla, description="Soruya göre analiz yapar ve LLM ile cevap üretir."),
    Tool(name="SesOzetPDF", func=ozet_ses_logs, description="`system_session_full.txt` dosyasındaki tüm ses kayıtlarını özetler ve PDF olarak kaydeder.")
]

for tool in tools:
    tool.func = track_tool_call(tool.func, tool.name)

system_prompt = """
Sen bir eğitim koçusun.
- Sayısal metrik kullanma, sadece nitel ifadeler (çok yüksek/orta/düşük) kullan.
- Konuşman sıcakkanlı, destekleyici ve motive edici olsun.
- Gerekirse araçları (Tools) çağır.
"""

agent = initialize_agent(
    tools=tools,
    llm=LLM,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    system_message=system_prompt
)

C:\Users\Hp\AppData\Local\Temp\ipykernel_22408\3114671232.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


----------------------------------------------------------------------------------------------------------------------

In [7]:
# 5) Yardımcı fonksiyonlar

def get_google_api_keys():
    keys = []
    idx = 1
    while True:
        key = os.getenv(f"GOOGLE_API_KEY_{idx}")
        if not key:
            break
        keys.append(key)
        idx += 1
    # Eski tip anahtar varsa onu da sona ekle
    legacy = os.getenv("GOOGLE_API_KEY")
    if legacy and legacy not in keys:
        keys.append(legacy)
    return keys

def try_load_llm_and_embedding():
    errors = []
    for api_key in get_google_api_keys():
        os.environ["GOOGLE_API_KEY"] = api_key  # Aktif anahtar olarak ayarla
        try:
            llm = GoogleGenerativeAI(model="gemini-1.5-flash-8b-001", temperature=0.1)
            embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
            print(f"✅ Google AI modelleri başarıyla yüklendi: {api_key[:8]}***")
            return llm, embedding
        except Exception as e:
            print(f"❌ API anahtarı başarısız: {api_key[:8]}*** => {e}")
            errors.append(f"{api_key[:8]}***: {e}")
            continue
    raise RuntimeError("Tüm Google API anahtarları denendi, hiçbiri çalışmadı:\n" + "\n".join(errors))



def get_safe_json(url):
    """Verilen URL’den güvenli şekilde JSON döndürür."""
    try:
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200 and resp.text.strip():
            return resp.json()
    except Exception:
        pass
    return {}


def kvalitatif_deger(score):
    """Verilen skor için nitel bir değerlendirme döndürür."""
    if score is None:
        return "ölçülemediği"
    if score >= 0.8:
        return "çok yüksek"
    if score >= 0.5:
        return "orta"
    return "düşük"


----------------------------------------------------------------------------------------------------------------------

In [ ]:
# 6) Flask API uygulaması

app = Flask(__name__)
CORS(app)

last_agent_response = "Henüz agent çalışmadı."
last_alert = None

@app.route("/active_tools", methods=["GET"])
def list_active_tools():
    """O anda çalışan araçları thread-safe şekilde döndürür."""
    with active_tools_lock:
        return jsonify({"active_tools": list(active_tools)})

@app.route("/download_session_summary", methods=["GET"])
def download_session_summary():
    """Hazırlanan özet PDF’i indirir."""
    pdf_path = "session_summary.pdf"
    if not os.path.exists(pdf_path):
        return jsonify({
            "error": "Özet PDF henüz oluşturulmadı. Agent'a SesOzetPDF aracını çalıştır deyin."
        }), 404
    return send_file(pdf_path, as_attachment=True)


@app.route("/last_response", methods=["GET"])
def get_last_response():
    """Son agent cevabını döndürür."""
    response = {
        "answer": last_agent_response
    }
    if last_alert is not None:
        response["alert"] = last_alert
    return jsonify(response)


@app.route("/ask", methods=["POST"])
def ask_agent():
    """Kullanıcıdan gelen soruları işleyerek agent cevapları döndürür."""
    global last_agent_response, last_alert
    try:
        data = request.get_json() or {}
        soru = data.get("question", "").strip()
        if not soru:
            return jsonify({"error": "Soru boş olamaz."}), 400

        session_id = datetime.now().strftime("%Y%m%d_%H")
        now_ts = datetime.now().isoformat()

        # Kullanıcı mesajını kaydet
        try:
            vectordb.add_texts(
                [soru],
                metadatas=[{
                    "user_id": STUDENT_ID,
                    "session_id": session_id,
                    "timestamp": now_ts,
                    "type": "conversation",
                    "speaker": "User"
                }]
            )
        except Exception as e:
            print(f"Veritabanı kayıt hatası: {e}")

        # Önceki 5 sohbeti getir (filtre güncel)
        try:
            conv_docs = vectordb.similarity_search(
                soru, k=5,
                filter={
                    "$and": [
                        {"user_id": STUDENT_ID},
                        {"type": "conversation"}
                    ]
                }
            )
            history = "\n".join(
                f"{d.metadata.get('speaker', 'User')}: {d.page_content}"
                for d in conv_docs
            )
        except Exception as e:
            print(f"Geçmiş sohbet alma hatası: {e}")
            history = ""

        # Geçmişle birlikte prompt oluştur
        prompt = f"""
Önceki konuşmalar:
{history}

Kullanıcı: {soru}
"""
        yanit = agent.run(prompt)
        last_agent_response = yanit
        return jsonify({"answer": yanit})
    except Exception as e:
        print(f"Genel API hatası: {e}")
        return jsonify({
            "answer": "Teknik bir sorun oluştu. Lütfen daha sonra tekrar deneyin.",
            "error": str(e)
        }), 500


# Flask sunucusunu ayrı bir thread'de başlat
def start_flask_server():
    try:
        app.run(host="0.0.0.0", port=8005, debug=False, threaded=True)
    except Exception as e:
        print(f"Flask sunucu başlatma hatası: {e}")


flask_thread = threading.Thread(target=start_flask_server, daemon=True)
flask_thread.start()

print("🚀 Flask API sunucusu başlatıldı: http://localhost:8005")

🚀 Flask API sunucusu başlatıldı: http://localhost:8005


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8005
 * Running on http://192.168.0.19:8005
Press CTRL+C to quit
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:49:45] "GET /active_tool

⚠️ Dikkatin dağıldı! Lütfen odaklan.
[Çerçeve RENGİ: KIRMIZI - ODAK YOK]


127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:01] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:50:02] "GET /active_tools HTTP/1.1" 200 -


...

127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:14] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:15] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:15] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:15] "GET /active_tools HTTP/1.1" 200 -
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 Internal error encountered..
127.0.0.1 - - [06/Aug/2025 22:51:15] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 2

Genel API hatası: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 13
}
]


127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "OPTIONS /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:51:45] "GET /active_tools HTTP/1.1" 200 -
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"

🔁 Öğrenme tarzı için yeterli veri yok.


127.0.0.1 - - [06/Aug/2025 22:52:23] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:23] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:23] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:24] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:25] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:25] "GET /last_response HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:25] "GET /active_tools HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 22:52:25] "GET /active_tools HTTP/

----------------------------------------------------------------------------------------------------------------------

In [9]:
# 7) Ana döngü: sensör verilerini topla ve periyodik uyarı ver

last_alert_time = 0
print("🚀 AI Agent ana döngüsü başlatılıyor...")

while True:
    try:
        now = time.time()

        # Her saniye: yeni sensör verisini çek, ChromaDB'ye kaydet
        data = get_safe_json(ENDPOINT_ATTENTION)
        keyboard_data = get_safe_json(ENDPOINT_KYBRD_MOUSE_INTERRUPT)
        script_data = get_safe_json(ENDPOINT_SCRIPT)

        # focus_score sadece hem user_texts hem de system_texts varsa alınır
        raw_user = script_data.get("user_texts", [])
        raw_system = script_data.get("system_texts", [])
        if raw_user and raw_system:
            focus_score = script_data.get("focus_score", None)
        else:
            focus_score = None

        metadata = {
            "user_id": STUDENT_ID,
            "timestamp": datetime.now().isoformat(),
            "type": "sensor",
            "attention": data.get("attention", 0),
            "screen": data.get("head_looking_at_screen", False),
            "focus_score": focus_score,
            "keyboard_activity": keyboard_data.get("keyboard_activity", False),
            "mouse_activity": keyboard_data.get("mouse_activity", False),
        }

        try:
            vectordb.add_texts(
                ["<sensor data>"],
                metadatas=[metadata]
            )
        except Exception as e:
            print(f"[Veri Kayıt Hatası] {e}")

 # Her 60 saniyede bir: son dakikalık tüm 'sensor' kayıtlarını çek, özetle ve uyarı ver
        if now - last_alert_time >= 60:
            one_min_ago = datetime.now() - timedelta(minutes=1)
            coll = vectordb._collection.get()
            recent_records = []

            for meta in coll["metadatas"]:
                ts_str = meta.get("timestamp")
                try:
                    ts = datetime.fromisoformat(ts_str)
                except Exception:
                    continue
                if ts >= one_min_ago and meta.get("type") == "sensor":
                    recent_records.append(meta)

            if recent_records:
                # --- YENİ BLOK: Önce küçük model ile özetle ---
                small_summary = summarize_records_with_small_llm(recent_records)
                # --- Sonra bu özetle büyük LLM'yi motive edici prompt ile çağır ---
                prompt = f"""
Aşağıda, son 1 dakikadaki sensör kayıtlarının küçük modelle çıkarılmış özeti var:

{small_summary}

Bu özet üzerinden:
- Kısa ve motive edici bir koç mesajı üret,
- Nitel yorumlarla özetle (çok iyi/orta/düşük gibi),
- Kullanıcıyı sıcak bir dille teşvik et.
"""
                yanit = LLM.invoke(prompt)
                print("\n🗨️ Uyarı:", yanit)
            else:
                print("🔁 Son 1 dakikada sensör verisi bulunamadı.")

            # --- HER 60 SN'DE AGENT'A OTOMATİK TÜRKÇE SORU SOR ---
            try:
                data = {"question": "Dikkat seviyemi analiz et ve gerekiyorsa uyarı ver. Lütfen Türkçe cevapla."}
                r = requests.post("http://localhost:8005/ask", json=data)
                resp = r.json()
                print("\n🤖 Agent’ın Türkçe cevabı:", resp.get("answer"))
            except Exception as e:
                print("Agent otomatik Türkçe soru sırasında hata:", e)

            last_alert_time = now

        # Döngünün çok hızlı dönmemesi için kısa uyku
        time.sleep(1)

    except KeyboardInterrupt:
        print("\n🛑 Program durduruldu (Ctrl+C)")
        break
    except Exception as e:
        print(f"🚨 Ana döngü hatası: {e}")
        time.sleep(5)
        continue

🚀 AI Agent ana döngüsü başlatılıyor...


C:\Users\Hp\AppData\Local\Temp\ipykernel_22408\3552124193.py:28: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = small_llm(prompt)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🗨️ Uyarı: **Koç Mesajı:**

Merhaba öğrenci_001!  Son bir dakikada dikkat seviyen çok iyi (0.77)! Fare etkinliğin de var, bu da odaklanmanıza ve çalışmanıza yardımcı oluyor.  Devam et!  Başarıya giden yolda harika bir ivme yakaladın.  Bu enerjiyle harika işler başaracaksın!


**Nitel Yorumlar:**

* **Dikkat:** Çok İyi
* **Fare Etkinliği:** İyi
* **Klavye Etkinliği:** Düşük (aktif değil)
* **Ekran:** Aktif (çalışma devam ediyor)


**Ek Not:**

Öğrenci_001'i, dikkatini ve çalışmasını sürdürmeye teşvik eden, motive edici ve sıcak bir dil kullanılmıştır.  Verilen bilgiler doğrultusunda, olumlu bir geri bildirim sağlanmıştır.


🤖 Agent’ın Türkçe cevabı: Teknik bir sorun oluştu. Lütfen daha sonra tekrar deneyin.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🗨️ Uyarı: **Koç Mesajı:**

Merhaba öğrenci_001!  Son bir dakikada ekranında aktif olduğunu ve fare hareketleri yaptığını görüyorum.  Dikkat seviyen de oldukça iyi!  Bu devam eden odaklanma ve etkinlik, harika bir öğrenme deneyimi için mükemmel bir başlangıç.  Devam et!  Başarıya giden yolda harika bir adım attın!


**Nitel Özet:**

* **Dikkat:** Çok İyi (0.81)
* **Klavyeyle Etkileşim:** Düşük (false)
* **Fare Etkileşimi:** İyi (true)
* **Ekran Etkinliği:** İyi (true)


**Ek Not:**

Öğrenci_001'e,  odaklanma ve etkinlik seviyesini koruması için olumlu bir geri bildirim ve teşvik sağlanmıştır.  Mesaj, öğrenciyi motive edici ve sıcak bir dille yazılmıştır.


🤖 Agent’ın Türkçe cevabı: Teknik bir sorun oluştu. Lütfen daha sonra tekrar deneyin.

🛑 Program durduruldu (Ctrl+C)


----------------------------------------------------------------------------------------------------------------------